### setup

In [1]:
import os
import argparse
import pickle
import time

import numpy as np; np.seterr(invalid='ignore')
import pandas as pd
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.data import TensorDataset, DataLoader

In [2]:
parser = {
    'data_path': '../data/wttsf/',
    'train_file': 'train_1.csv',
    'intermediate_path': '../intermediate/',
    'n_epoch': 3,
    'future': 73,
    'batch_size': 64,
    'hidden_size': 128,
    'log_every': 10,
    'read_from_file': False,
    'train': True,
    'model_name': '',
    'cuda': True,
    'seed': 20170903,
}
args = argparse.Namespace(**parser)

args.cuda = args.cuda and torch.cuda.is_available()
torch.manual_seed(args.seed)

args.intermediate_path = os.path.join(args.intermediate_path, str(args.seed))

### model

In [3]:
class DenseLSTMForecast(nn.Module):
    def __init__(self, hidden_size):
        super(DenseLSTMForecast, self).__init__()
        self.lstm1 = nn.LSTMCell(15, hidden_size, bias=False)
        self.lstm2 = nn.LSTMCell(hidden_size+15, hidden_size, bias=False)
        self.lstm3 = nn.LSTMCell(2*hidden_size+15, hidden_size, bias=False)
        self.linear = nn.Linear(3*hidden_size+15, 1)
        self.hidden_size = hidden_size

    def forward(self, x, feature, future=1):
        o = []
        tt = torch.cuda if args.cuda else torch
        h1_t = Variable(tt.FloatTensor(x.size(0), self.hidden_size).zero_())
        c1_t = Variable(tt.FloatTensor(x.size(0), self.hidden_size).zero_())
        h2_t = Variable(tt.FloatTensor(x.size(0), self.hidden_size).zero_())
        c2_t = Variable(tt.FloatTensor(x.size(0), self.hidden_size).zero_())
        h3_t = Variable(tt.FloatTensor(x.size(0), self.hidden_size).zero_())
        c3_t = Variable(tt.FloatTensor(x.size(0), self.hidden_size).zero_())
        
        for x_t in x.chunk(x.size(1), dim=1):
            x_t = x_t.squeeze(dim=1)
            xd_t = torch.cat([x_t, feature], dim=1)
            h1_t, c1_t = self.lstm1(xd_t, (h1_t, c1_t))
            h1d_t = torch.cat([xd_t, h1_t], dim=1)
            h2_t, c2_t = self.lstm2(h1d_t, (h2_t, c2_t))
            h2d_t = torch.cat([xd_t, h1_t, h2_t], dim=1)
            h3_t, c3_t = self.lstm3(h2d_t, (h3_t, c3_t))
            h3d_t = torch.cat([xd_t, h1_t, h2_t, h3_t], dim=1)
            o_t = self.linear(h3d_t)
            o.append(o_t)
            
        for i in range(future-1):
            od_t = torch.cat([o_t, feature], dim=1)
            h1_t, c1_t = self.lstm1(od_t, (h1_t, c1_t))
            h1d_t = torch.cat([od_t, h1_t], dim=1)
            h2_t, c2_t = self.lstm2(h1d_t, (h2_t, c2_t))
            h2d_t = torch.cat([od_t, h1_t, h2_t], dim=1)
            h3_t, c3_t = self.lstm3(h2d_t, (h3_t, c3_t))
            h3d_t = torch.cat([od_t, h1_t, h2_t, h3_t], dim=1)
            o_t = self.linear(h3d_t)
            o.append(o_t)

        return torch.stack(o, dim=1)

### utils

In [4]:
def smape(y_pred, y_true):
    y_pred = np.around(np.clip(np.exp(y_pred)-1, 0, None))
    y_true = np.around(np.exp(y_true) - 1)
    raw_smape = np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred))
    kaggle_smape = np.nan_to_num(raw_smape)
    return np.mean(kaggle_smape) * 200

In [5]:
def get_data():
#     raw_data_file = os.path.join(args.intermediate_path,
#                                  'raw_data.pkl')
    scaled_data_file = os.path.join(args.intermediate_path,
                                    'scaled_data.pkl')
    scaler_file = os.path.join(args.intermediate_path, 'scaler.pkl')
    features_file = os.path.join(args.intermediate_path, 'features.pkl')
    
    if not args.read_from_file:
        data_df = pd.read_csv(os.path.join(args.data_path, args.train_file),
                              index_col='Page')
        data_df["agent"] = data_df.index.str.rsplit('_').str.get(-1)
        data_df["access"] = data_df.index.str.rsplit('_').str.get(-2)
        data_df["project"] = data_df.index.str.rsplit('_').str.get(-3)
        features = pd.get_dummies(data_df[["agent", "access", "project"]],
            columns=["agent", "access", "project"]).values.astype('float32')
        raw_data = np.nan_to_num(
            data_df.iloc[:,:-3].values.astype('float32'))
        data = np.log1p(raw_data)
        scaler = StandardScaler()
        scaler.fit(np.swapaxes(data[:, :-args.future], 0, 1))
        scaled_data = scaler.transform(np.swapaxes(data, 0, 1))
        scaled_data = np.swapaxes(scaled_data, 0, 1)
        
#         with open(raw_data_file, 'wb') as f:
#             pickle.dump(raw_data, f)
        with open(scaled_data_file, 'wb') as f:
            pickle.dump(scaled_data, f)
        with open(scaler_file, 'wb') as f:
            pickle.dump(scaler, f)
        with open(features_file, 'wb') as f:
            pickle.dump(features, f)
    else:
#         with open(raw_data_file, 'rb') as f:
#             raw_data = pickle.load(f)
        with open(scaled_data_file, 'rb') as f:
            scaled_data = pickle.load(f)
        with open(scaler_file, 'rb') as f:
            scaler = pickle.load(f)
        with open(features_file, 'rb') as f:
            features = pickle.load(f)
    return scaled_data, scaler, features

In [6]:
def train(scaled_data, scaler, features, model, criterion, optimizer):
    p = np.random.permutation(scaled_data.shape[0])
    inverse_p = np.argsort(p)

    input_tensor = torch.from_numpy(scaled_data[p, :-1]).unsqueeze(2)
    target_tensor = torch.from_numpy(scaled_data[p, 1:]).unsqueeze(2)
    features_tensor = torch.from_numpy(features[p, :])
    dataset = TensorDataset(input_tensor, target_tensor)
    data_loader = DataLoader(dataset, args.batch_size)
    
    train_loss = 0
    val_output_list = []
    init_time = time.time()
    for i, (inputt, target) in enumerate(data_loader):
        feature = features_tensor[i*args.batch_size:(i*args.batch_size
                                                     +inputt.size(0))]
        if args.cuda:
            inputt = inputt.cuda()
            target = target.cuda()
            feature = feature.cuda()
        inputt = Variable(inputt)
        target = Variable(target)
        feature = Variable(feature)
        
        output = model(inputt, feature)
        pos = np.random.randint(args.future,
                                output.size(1)-2*args.future+1)
#        pos_val = np.random.randint(args.future,
#                                    output.size(1)-args.future+1)
        loss = criterion(output[:, pos:pos+args.future],
                         target[:, pos:pos+args.future])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.data[0] * inputt.size(0)
        
        val_output_list.append(output[:, -args.future:]
                               .data.squeeze(2).cpu().numpy())

        if i % args.log_every == 0:

            print("   % Time: {:4.0f}s | Batch: {:4} | "
                  "Train loss: {:.4f}".format(
                      time.time()-init_time, i+1, loss.data[0]))
        
    val_output_all = np.concatenate(val_output_list, axis=0)[inverse_p]
    prediction = np.swapaxes(scaler.inverse_transform(
            np.swapaxes(val_output_all, 0, 1)), 0, 1)
    
    var_target = np.swapaxes(scaler.inverse_transform(
            np.swapaxes(scaled_data[:, -args.future:], 0, 1)), 0, 1)
    train_loss /= scaled_data.shape[0]
    val_loss = smape(prediction, var_target)
    
    print("="*10)
    print("   % Epoch: {} | Time: {:4.0f}s | "
          "Train loss: {:.4f} | Val loss: {:.4f}"
          .format(epoch, time.time()-init_time, train_loss , val_loss))
    print("="*10)
    return val_loss

#             val_output = output[
#                 :, pos_val:pos_val+args.future].data.squeeze(2).cpu().numpy()
#             val_target = target[
#                 :, pos_val:pos_val+args.future].data.squeeze(2).cpu().numpy()
#             val_scale = scaler.scale_[p][
#                 i*args.batch_size:i*args.batch_size+inputt.size(0)]
#             val_mean = scaler.mean_[p][
#                 i*args.batch_size:i*args.batch_size+inputt.size(0)]
#             raw_val_output = (val_output.T * val_scale + val_mean).T
#             raw_val_target = (val_target.T * val_scale + val_mean).T
#             val_loss = smape(raw_val_output, raw_val_target)


In [7]:
def forecast(scaled_data, scaler, features, model):
    input_tensor = torch.from_numpy(scaled_data).unsqueeze(2)
    target_tensor = torch.zeros(input_tensor.size(0))
    features_tensor = torch.from_numpy(features)
    dataset = torch.utils.data.TensorDataset(input_tensor, target_tensor)
    data_loader = DataLoader(dataset, args.batch_size)
    
    output_list = []
    for i, (inputt, _) in enumerate(data_loader):
        feature = features_tensor[i*args.batch_size:(i*args.batch_size
                                                     +inputt.size(0))]
        if args.cuda:
            inputt = inputt.cuda()
            feature = feature.cuda()
        inputt = Variable(inputt)
        feature = Variable(feature)
        output = model(inputt, feature, args.future)
        output_list.append(output.data.squeeze(2).cpu().numpy()
                           [:, -args.future:])
        
    output_all = np.concatenate(output_list, axis=0)
    prediction = np.swapaxes(scaler.inverse_transform(
            np.swapaxes(output_all, 0, 1)), 0, 1)
    prediction = np.around(np.clip(np.exp(prediction) - 1, 0, None))
    return prediction

In [8]:
def save_model(model, epoch, loss):
    model_file = os.path.join(args.intermediate_path,
                              "model_seed{}_epoch{}_loss_{:.4f}.pth"
                              .format(args.seed, epoch, loss))
    torch.save(model.state_dict(), os.path.join(model_file))

### prepare

In [9]:
scaled_data, scaler, features = get_data()

In [10]:
model = DenseLSTMForecast(args.hidden_size)
if args.cuda:
    model.cuda()
criterion = nn.L1Loss()

In [11]:
optimizer = optim.Adam(model.parameters())
scheduler = MultiStepLR(optimizer, milestones=[2])

### train

In [12]:
if args.train:
    for epoch in range(1, args.n_epoch+1):
        print("=> EPOCH {}".format(epoch))
        scheduler.step()
        val_loss = train(scaled_data, scaler, features,
                         model, criterion, optimizer)
#        scheduler.step(val_loss)
#        save_model(model, epoch, val_loss)
else:
    model_file = os.path.join(args.intermediate_path, args.model_name)
    model.load_state_dict(torch.load(model_file))

=> EPOCH 1
   % Time:    1s | Batch:    1 | Train loss: 0.7365
   % Time:   10s | Batch:   11 | Train loss: 0.4983
   % Time:   19s | Batch:   21 | Train loss: 0.4348
   % Time:   29s | Batch:   31 | Train loss: 0.4087
   % Time:   39s | Batch:   41 | Train loss: 0.4330
   % Time:   48s | Batch:   51 | Train loss: 0.4578
   % Time:   57s | Batch:   61 | Train loss: 0.4657
   % Time:   67s | Batch:   71 | Train loss: 0.4188
   % Time:   77s | Batch:   81 | Train loss: 0.4300
   % Time:   86s | Batch:   91 | Train loss: 0.4873
   % Time:   95s | Batch:  101 | Train loss: 0.4488
   % Time:  105s | Batch:  111 | Train loss: 0.4561
   % Time:  114s | Batch:  121 | Train loss: 0.4403
   % Time:  125s | Batch:  131 | Train loss: 0.4372
   % Time:  134s | Batch:  141 | Train loss: 0.4042
   % Time:  143s | Batch:  151 | Train loss: 0.4476
   % Time:  153s | Batch:  161 | Train loss: 0.4459
   % Time:  162s | Batch:  171 | Train loss: 0.4061
   % Time:  172s | Batch:  181 | Train loss: 0.3853
 

   % Time: 1367s | Batch: 1581 | Train loss: 0.3800
   % Time: 1375s | Batch: 1591 | Train loss: 0.3610
   % Time: 1383s | Batch: 1601 | Train loss: 0.3804
   % Time: 1390s | Batch: 1611 | Train loss: 0.3785
   % Time: 1397s | Batch: 1621 | Train loss: 0.4012
   % Time: 1405s | Batch: 1631 | Train loss: 0.3787
   % Time: 1412s | Batch: 1641 | Train loss: 0.3699
   % Time: 1420s | Batch: 1651 | Train loss: 0.4594
   % Time: 1427s | Batch: 1661 | Train loss: 0.3835
   % Time: 1435s | Batch: 1671 | Train loss: 0.4014
   % Time: 1442s | Batch: 1681 | Train loss: 0.4029
   % Time: 1450s | Batch: 1691 | Train loss: 0.4172
   % Time: 1457s | Batch: 1701 | Train loss: 0.3888
   % Time: 1465s | Batch: 1711 | Train loss: 0.4061
   % Time: 1472s | Batch: 1721 | Train loss: 0.4182
   % Time: 1480s | Batch: 1731 | Train loss: 0.3828
   % Time: 1487s | Batch: 1741 | Train loss: 0.3538
   % Time: 1495s | Batch: 1751 | Train loss: 0.3734
   % Time: 1502s | Batch: 1761 | Train loss: 0.4713
   % Time: 1

   % Time:  638s | Batch:  871 | Train loss: 0.3855
   % Time:  646s | Batch:  881 | Train loss: 0.4241
   % Time:  653s | Batch:  891 | Train loss: 0.3402
   % Time:  660s | Batch:  901 | Train loss: 0.3807
   % Time:  667s | Batch:  911 | Train loss: 0.4179
   % Time:  675s | Batch:  921 | Train loss: 0.3713
   % Time:  682s | Batch:  931 | Train loss: 0.3670
   % Time:  689s | Batch:  941 | Train loss: 0.3918
   % Time:  697s | Batch:  951 | Train loss: 0.3742
   % Time:  704s | Batch:  961 | Train loss: 0.3800
   % Time:  711s | Batch:  971 | Train loss: 0.3955
   % Time:  718s | Batch:  981 | Train loss: 0.3737
   % Time:  725s | Batch:  991 | Train loss: 0.3643
   % Time:  732s | Batch: 1001 | Train loss: 0.3373
   % Time:  739s | Batch: 1011 | Train loss: 0.3692
   % Time:  746s | Batch: 1021 | Train loss: 0.3971
   % Time:  753s | Batch: 1031 | Train loss: 0.4073
   % Time:  760s | Batch: 1041 | Train loss: 0.3858
   % Time:  767s | Batch: 1051 | Train loss: 0.3522
   % Time:  

   % Time:  112s | Batch:  161 | Train loss: 0.4009
   % Time:  119s | Batch:  171 | Train loss: 0.3378
   % Time:  126s | Batch:  181 | Train loss: 0.3817
   % Time:  133s | Batch:  191 | Train loss: 0.3618
   % Time:  139s | Batch:  201 | Train loss: 0.3581
   % Time:  146s | Batch:  211 | Train loss: 0.3557
   % Time:  153s | Batch:  221 | Train loss: 0.3771
   % Time:  160s | Batch:  231 | Train loss: 0.3754
   % Time:  167s | Batch:  241 | Train loss: 0.3881
   % Time:  174s | Batch:  251 | Train loss: 0.3618
   % Time:  181s | Batch:  261 | Train loss: 0.3970
   % Time:  188s | Batch:  271 | Train loss: 0.3983
   % Time:  195s | Batch:  281 | Train loss: 0.3637
   % Time:  202s | Batch:  291 | Train loss: 0.3754
   % Time:  209s | Batch:  301 | Train loss: 0.3644
   % Time:  216s | Batch:  311 | Train loss: 0.4070
   % Time:  222s | Batch:  321 | Train loss: 0.3551
   % Time:  229s | Batch:  331 | Train loss: 0.4038
   % Time:  236s | Batch:  341 | Train loss: 0.3563
   % Time:  

   % Time: 1209s | Batch: 1741 | Train loss: 0.4009
   % Time: 1216s | Batch: 1751 | Train loss: 0.3886
   % Time: 1223s | Batch: 1761 | Train loss: 0.3946
   % Time: 1230s | Batch: 1771 | Train loss: 0.3401
   % Time: 1237s | Batch: 1781 | Train loss: 0.3732
   % Time: 1244s | Batch: 1791 | Train loss: 0.3608
   % Time: 1251s | Batch: 1801 | Train loss: 0.4054
   % Time: 1258s | Batch: 1811 | Train loss: 0.3544
   % Time: 1265s | Batch: 1821 | Train loss: 0.4597
   % Time: 1272s | Batch: 1831 | Train loss: 0.4039
   % Time: 1279s | Batch: 1841 | Train loss: 0.4299
   % Time: 1286s | Batch: 1851 | Train loss: 0.3508
   % Time: 1293s | Batch: 1861 | Train loss: 0.4037
   % Time: 1300s | Batch: 1871 | Train loss: 0.3345
   % Time: 1307s | Batch: 1881 | Train loss: 0.3725
   % Time: 1314s | Batch: 1891 | Train loss: 0.3677
   % Time: 1321s | Batch: 1901 | Train loss: 0.4089
   % Time: 1328s | Batch: 1911 | Train loss: 0.3560
   % Time: 1335s | Batch: 1921 | Train loss: 0.3430
   % Time: 1

### test

In [13]:
prediction = forecast(scaled_data, scaler, features, model)
prediction_file = os.path.join(args.intermediate_path,
                               'prediction_seed{}.pkl'.format(args.seed))

with open(prediction_file, 'wb') as f:
    pickle.dump(prediction, f)